In [1]:
#install the pandas library

!pip install pandas pandasql


In [2]:
import pandas as pd
import pandasql as ps

In [3]:
# read in the csv file as a pandas dataframe
reservations = pd.read_csv("reservations.csv")
merchants = pd.read_csv("merchants.csv")

reservations.head()

,reservation_id,reservation_created_date,reservation_completed_date,reservation_status,customer_id,merchant_id,revenue,marketing_channel,country
0,b4ea-a5cb0-10e4-90a4,2023-06-22 10:08:19 UTC,2023-06-24 06:45:00 UTC,successful,2cbc-c40df-c6f4-b9d6,511d-d4351-7704-a5b1,4.219409,Affiliate,NZ
1,14d4-4e910-1144-a32a,2023-06-22 21:31:03 UTC,2023-06-23 09:30:00 UTC,successful,1775-52029-4195-b8ae,f4a5-5c44c-611b-8d48,11.251758,Direct,NZ
2,c837-79fa0-10ce-a727,2023-06-22 07:31:18 UTC,2023-06-24 05:45:00 UTC,canceled,8968-850e2-844f-811b,f4a5-5c44c-611b-8d48,5.625879,Referral,NZ
3,b362-28100-10e0-a21c,2023-06-22 09:39:37 UTC,2023-06-24 07:00:00 UTC,successful,1b97-7d46a-bef4-a20e,f4a5-5c44c-611b-8d48,4.219409,Other,NZ
4,d61c-ca6e0-114d-aa03,2023-06-22 22:40:52 UTC,2023-06-23 00:30:00 UTC,successful,9be2-22a6a-dcd5-aa37,4bea-ab9e0-ad2d-8075,7.876231,Other,NZ


#### Exploratory Data Analysis

In [4]:
# Checks if a city name exists in more than one country
query = """
   SELECT m.city, COUNT(DISTINCT r.country) AS num_countries
   FROM reservations r
   JOIN merchants m ON r.merchant_id = m.merchant_id
   GROUP BY m.city
   HAVING num_countries > 1;
"""

# Execute the query using pandasql
result = ps.sqldf(query, locals())

# Display the result
print(result)

           city  num_countries
0  Christchurch              2
1     Newcastle              2
2         Perth              2


In [5]:
# Check if reservation_id values are unique 
query = """
    SELECT r.reservation_id,
           COUNT(r.reservation_id) as reservation_count
    FROM reservations r
    JOIN merchants m ON r.merchant_id = m.merchant_id
    GROUP BY reservation_id 
    HAVING reservation_count > 1;
"""

# Execute the query using pandasql
result = ps.sqldf(query, locals())

# Display the result
print(result)

Empty DataFrame
Columns: [reservation_id, reservation_count]
Index: []


#### Data Analysis

In [6]:
# Define the SQL query to identify the most important cities using different metrics

# Total Revenue in each city
query = """
    SELECT m.city,
           r.country,
           ROUND(SUM(r.revenue),2) as total_revenue
    FROM reservations r
    JOIN merchants m ON r.merchant_id = m.merchant_id
    GROUP BY m.city, r.country
    ORDER BY total_revenue DESC
    LIMIT 5; 
"""

# Execute the query using pandasql
result = ps.sqldf(query, locals())

# Display the result
print(result)


        city country  total_revenue
0  Singapore      SG     1365393.20
1       Wien      AT      347852.70
2     Berlin      DE      338239.80
3  Melbourne      AU      331125.56
4     London      GB      303714.16


In [7]:
# Total number of reservations in each city
query = """
    SELECT m.city,
           r.country,
           COUNT(r.reservation_id) as total_reservations
    FROM reservations r
    JOIN merchants m ON r.merchant_id = m.merchant_id
    GROUP BY m.city, r.country
    ORDER BY total_reservations DESC
    LIMIT 5; 
"""

# Execute the query using pandasql
result = ps.sqldf(query, locals())

# Display the result
print(result)

        city country  total_reservations
0  Singapore      SG              369658
1       Wien      AT              116550
2  Melbourne      AU               71245
3     Berlin      DE               63102
4       Roma      IT               58655


In [8]:
# Total number of merchants in each city
query = """
    SELECT m.city,
           r.country,
           COUNT(DISTINCT r.merchant_id) as total_merchants
    FROM reservations r
    JOIN merchants m ON r.merchant_id = m.merchant_id
    GROUP BY m.city, r.country
    ORDER BY total_merchants DESC
    LIMIT 5; 
"""

# Execute the query using pandasql
result = ps.sqldf(query, locals())

# Display the result
print(result)

        city country  total_merchants
0  Singapore      SG             1311
1       Roma      IT              964
2     London      GB              760
3       Wien      AT              649
4     Berlin      DE              632


In [9]:
# Total number of customers in each city
query = """
    SELECT m.city, 
           r.country,
           COUNT(DISTINCT r.customer_id) as total_customers
    FROM reservations r
    JOIN merchants m ON r.merchant_id = m.merchant_id
    GROUP BY m.city, r.country
    ORDER BY total_customers DESC
    LIMIT 5; 
"""

# Execute the query using pandasql
result = ps.sqldf(query, locals())

# Display the result
print(result)

        city country  total_customers
0  Singapore      SG           226359
1       Wien      AT            79471
2  Melbourne      AU            58232
3     Berlin      DE            53521
4       Roma      IT            50903


In [10]:
# Export the csv file to be used for data visualizations in Tableau
query = """
    SELECT r.*, m.city, m.cuisine
    FROM reservations r
    JOIN merchants m ON r.merchant_id = m.merchant_id
"""

result = ps.sqldf(query, locals())

result.to_csv('joined_result.csv', index=False)

